In [2]:
import torch
import torch.nn.functional as F
from torch import nn, Tensor
from torch.nn import TransformerEncoder, TransformerEncoderLayer
import numpy as np
import random, math
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import r2_score
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
import tensorflow as tf
from sklearn.model_selection import StratifiedKFold, KFold
# device = torch.device("cuda:{}".format(0) if torch.cuda.is_available() else "cpu")
device = torch.device("cpu")


In [3]:
def setup_seed(seed):
     torch.manual_seed(seed)
     torch.cuda.manual_seed_all(seed)
     np.random.seed(seed)
     random.seed(seed)
     torch.backends.cudnn.deterministic = True

seed = 1
setup_seed(seed)

In [21]:
data = pd.read_csv('new_27.csv')

max_seq_len1 = 6  # 7 (all-16) 6
max_seq_len2 = 5  # 4 (all-16) 5

data = data[(data['电共振峰对应位置'].notnull()) & (data['磁共振峰对应位置'].notnull()) & (data['minRL'].notnull())]

src_key_padding_mask1 = tf.keras.preprocessing.sequence.pad_sequences(
    data['电共振峰对应位置'].apply(lambda x: [False for i in str(x).split(',')] if x is not np.NaN else []), padding='post', value=True, maxlen=max_seq_len1, dtype=bool
)

src_key_padding_mask2 = tf.keras.preprocessing.sequence.pad_sequences(
    data['磁共振峰对应位置'].apply(lambda x: [False for i in str(x).split(',')] if x is not np.NaN else []), padding='post', value=True, maxlen=max_seq_len2, dtype=bool
)

e_features = ['电共振峰对应位置', '电共振峰对应位置_e', '电共振峰对应位置_ee', '电共振峰对应位置_m', '电共振峰对应位置_mm']
m_features = ['磁共振峰对应位置', '磁共振峰对应位置_e', '磁共振峰对应位置_ee', '磁共振峰对应位置_m', '磁共振峰对应位置_mm']

for col in e_features:
    data[col]  = tf.keras.preprocessing.sequence.pad_sequences(
        data[col].apply(lambda x: [float(i) for i in str(x).split(',')] if x is not np.NaN else []), padding="post", value=np.nan, maxlen=max_seq_len1, dtype=float
    ).tolist()

for col in m_features:
    data[col]  = tf.keras.preprocessing.sequence.pad_sequences(
        data[col].apply(lambda x: [float(i) for i in str(x).split(',')] if x is not np.NaN else []), padding="post", value=np.nan, maxlen=max_seq_len2, dtype=float
    ).tolist()

X1 = np.stack(map(lambda i: np.stack(data.loc[i, e_features]), data.index))
src_key_padding_mask1 = np.isnan(X1)[:, 0, :]
X1[np.isnan(X1)] = 0
X2 = np.stack(map(lambda i: np.stack(data.loc[i, m_features]), data.index))
src_key_padding_mask2 = np.isnan(X2)[:, 0, :]
X2[np.isnan(X2)] = 0
Y = data['minRL'].to_numpy().reshape(-1, 1)

In [11]:
print('X1.shape:', X1.shape)
print('src_key_padding_mask1.shape:', src_key_padding_mask1.shape)
print('X2.shape:', X2.shape)
print('src_key_padding_mask2.shape:', src_key_padding_mask2.shape)
print('Y.shape:', Y.shape)


X1.shape: (27, 5, 5)
src_key_padding_mask1.shape: (27, 5)
X2.shape: (27, 5, 5)
src_key_padding_mask2.shape: (27, 5)
Y.shape: (27, 1)


In [12]:
class MaterialDataset(Dataset):
    def __init__(self, X1, X2, Y, src_key_padding_mask1, src_key_padding_mask2):
        self.X1 = X1
        self.X2 = X2
        self.Y = Y
        self.src_key_padding_mask1 = src_key_padding_mask1
        self.src_key_padding_mask2 = src_key_padding_mask2

    def __getitem__(self, index):
        return self.X1[index], self.X2[index], self.Y[index], self.src_key_padding_mask1[index], self.src_key_padding_mask2[index]

    def __len__(self):
        return len(self.X1)

In [13]:
class PositionalEncoding(nn.Module):
    
    def __init__(self, d_model: int, dropout: float = 0.1, max_len: int = 5):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        position = torch.arange(max_len).unsqueeze(1)

        pe = torch.zeros(max_len, 1, d_model)

        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        div_term = torch.exp(torch.arange(1, d_model, 2) * (-math.log(10000.0) / d_model))
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)
        # print('pe shape:', pe.shape)

    def forward(self, x: Tensor) -> Tensor:
        """
        Args:
            x: Tensor, shape [seq_len, batch_size, embedding_dim]
        """
        x = x + self.pe[:x.size(0)]
        return self.dropout(x)


class TransformerModel(nn.Module):

    def __init__(self, ntoken: int, d_model: int, nhead: int, d_hid: int, max_len1: int, max_len2: int,
                 nlayers: int, dropout: float = 0.5):
        super().__init__()
        self.pos_encoder1 = PositionalEncoding(d_model, dropout, max_len=max_len1)
        self.transformer_encoder1 = TransformerEncoder(TransformerEncoderLayer(d_model, nhead, d_hid, dropout), nlayers)
        
        self.pos_encoder2 = PositionalEncoding(d_model, dropout, max_len=max_len2)
        self.transformer_encoder2 = TransformerEncoder(TransformerEncoderLayer(d_model, nhead, d_hid, dropout), nlayers)

        # self.decoder = nn.Linear(d_model * 5, ntoken) # 相加
        self.decoder = nn.Linear(d_model * (max_len1 + max_len2), ntoken) # 拼接

        self.init_weights()

    def init_weights(self) -> None:
        initrange = 0.1
        self.decoder.bias.data.zero_()
        self.decoder.weight.data.uniform_(-initrange, initrange)

    def forward(self, X1: Tensor, X2: Tensor, src_key_padding_mask1: Tensor, src_key_padding_mask2: Tensor) -> Tensor:

        src = self.pos_encoder1(X1)
        output1 = self.transformer_encoder1(src=src, src_key_padding_mask = src_key_padding_mask1)

        src = self.pos_encoder2(X2)
        output2 = self.transformer_encoder2(src=src, src_key_padding_mask = src_key_padding_mask2)
        
        output = torch.cat([output1.permute(1, 0, 2).flatten(1), output2.permute(1, 0, 2).flatten(1)], dim=1) # 拼接
        # output = output1.permute(1, 0, 2).flatten(1) + output2.permute(1, 0, 2).flatten(1) # 相加
        output = self.decoder(output)
        return output


def generate_square_subsequent_mask(sz: int) -> Tensor:
    """Generates an upper-triangular matrix of -inf, with zeros on diag."""
    return torch.triu(torch.ones(sz, sz) * float('-inf'), diagonal=1)


# 数据划分

In [14]:
def test(val_loader, model, device):
    model.eval()
    score = 0.0
    with torch.no_grad():
        for i, (x1, x2, y, padding_mask1, padding_mask2) in enumerate(val_loader):
            x1 = x1.float().to(device)
            x2 = x2.float().to(device)
            y = y.float()
            padding_mask1 = padding_mask1.float().to(device)
            padding_mask2 = padding_mask2.float().to(device)

            pre_y = model(x1.permute(2,0,1), x2.permute(2,0,1), padding_mask1, padding_mask2)
            pre_y = pre_y.detach().numpy()
            # print(pre_y.shape)
            score += max(0, r2_score(y, pre_y))
            
        score /= (i + 1)
    
    return score

In [24]:
# 新改模型定义
folds = 5
seed = 2020
kf = KFold(n_splits=folds, shuffle=True, random_state=seed)

kf_scores = []

for i, (train_index, valid_index) in enumerate(kf.split(X1)):
    print('*****************************************{}********************************'.format(i + 1))
    model = TransformerModel(ntoken=1, d_model=5, nhead=5, d_hid=16, max_len1=max_seq_len1, max_len2=max_seq_len2, nlayers=5 , dropout=0.2)
    model = model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    # criterion = nn.MSELoss()
    criterion = nn.SmoothL1Loss()

    X1_train, X2_train, y_train = X1[train_index], X2[train_index], Y[train_index]
    X1_val, X2_val, y_val = X1[valid_index], X2[valid_index], Y[valid_index]

    src_key_padding_mask1_train, src_key_padding_mask2_train = src_key_padding_mask1[train_index], src_key_padding_mask2[train_index]
    src_key_padding_mask1_val, src_key_padding_mask2_val = src_key_padding_mask1[valid_index], src_key_padding_mask2[valid_index]
    
    train_loader = DataLoader(dataset = MaterialDataset(X1_train, X2_train, y_train, src_key_padding_mask1_train, src_key_padding_mask2_train), batch_size=1000, shuffle=False, num_workers=0)
    val_loader = DataLoader(dataset = MaterialDataset(X1_val, X2_val, y_val, src_key_padding_mask1_val, src_key_padding_mask2_val), batch_size=1000, shuffle=False, num_workers=0)

    max_score = -1
    best_epoch = 0
    model.train()
    for epoch in range(15000):
        if epoch - best_epoch > 2000:
            print('early stopping!!!!')
            break
        for j, (x1, x2, y, padding_mask1, padding_mask2) in enumerate(train_loader):
            x1 = x1.float().to(device)
            x2 = x2.float().to(device)
            y = y.float().to(device)
            padding_mask1 = padding_mask1.float().to(device)
            padding_mask2 = padding_mask2.float().to(device)

            pre_y = model(x1.permute(2,0,1), x2.permute(2,0,1), padding_mask1, padding_mask2)

            loss = criterion(pre_y, y)

            # Backpropagation
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        
        score = test(val_loader, model, device)
        model.train()

        # print(f"epoch[{epoch}]: loss: {loss:>7f} score: {score:>7f}")
        
        if score > max_score:
            max_score = score
            best_epoch = epoch
            print(f"epoch[{epoch}]: loss: {loss:>7f} score: {score:>7f}")
            print(f"Higher score: {(max_score):>4f}")
            torch.save(model, f'minRL_{i}.pt')
    kf_scores.append(max_score)

*****************************************1********************************
epoch[0]: loss: 15.001664 score: 0.000000
Higher score: 0.000000
epoch[1104]: loss: 3.601116 score: 0.001531
Higher score: 0.001531
epoch[1105]: loss: 3.642489 score: 0.020683
Higher score: 0.020683
epoch[1106]: loss: 3.435272 score: 0.022794
Higher score: 0.022794
epoch[1116]: loss: 3.616039 score: 0.023556
Higher score: 0.023556
epoch[1117]: loss: 3.314700 score: 0.047268
Higher score: 0.047268
epoch[1118]: loss: 3.508588 score: 0.055319
Higher score: 0.055319
epoch[1123]: loss: 3.402287 score: 0.055393
Higher score: 0.055393
epoch[1152]: loss: 3.279778 score: 0.066654
Higher score: 0.066654
epoch[1153]: loss: 3.662340 score: 0.090920
Higher score: 0.090920
epoch[1154]: loss: 3.405418 score: 0.099556
Higher score: 0.099556
epoch[1155]: loss: 3.595795 score: 0.102047
Higher score: 0.102047
epoch[1158]: loss: 3.108091 score: 0.104625
Higher score: 0.104625
epoch[1159]: loss: 3.432725 score: 0.112709
Higher score

KeyboardInterrupt: 

In [25]:
print(np.mean(kf_scores))
print(kf_scores)
print(np.std(kf_scores))

0.28916521833402
[0.16379419329971, 0.21528940817405273, 0.2658270515948582, 0.5117502202674589]
0.13347685413340304


In [ ]:
# minRL new_27_limit_5
# seed == 2020
0.4009413858445585
[0.34387797551922605, 0.5383718305817475, 0.3205743514327021]
0.0976425797808438

In [ ]:
# minRL new_27_limit_3
# seed == 2020
0.3958051487992849
[0.48662754341797954, 0.5688517562348477, 0.13193614674502763]
0.18957908284261504

In [ ]:
# minRL new_27
# seed == 2020
0.23430058456247915
[0.012273413554796608, 0.5903697701056589, 0.10025857002698191]
0.25432825981382967